In [0]:
from keras import layers
from keras import models
from keras.models import Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, Flatten

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
RESOLUTION = 150
BATCH_SIZE = 256

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/deeplearning_cursoCienciaDados_Insight/notebooks/data/state-farm-distracted-driver-detection/',
        target_size=(RESOLUTION, RESOLUTION),
        class_mode='categorical', subset="training")

val_generator = val_datagen.flow_from_directory(
        '/content/drive/My Drive/deeplearning_cursoCienciaDados_Insight/notebooks/data/state-farm-distracted-driver-detection/',
        target_size=(RESOLUTION, RESOLUTION),
        class_mode='categorical', subset="validation")

Found 19066 images belonging to 10 classes.
Found 3358 images belonging to 10 classes.


### Instancie uma rede VGG16 do Keras utililizando como pesos os pre-treinados nos dados do ImageNet.

In [6]:
conv_base = VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
conv_base.summary()

W0716 17:57:24.808633 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 17:57:24.828539 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 17:57:24.832885 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0716 17:57:24.863605 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 2s 0us/step


W0716 17:57:27.485344 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0716 17:57:27.486974 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

### Utilizando a rede pre-treinada da questão anterior, crie um modelo usando esta rede aplicando a técnica de fine tunning para as 10 classes do problema de detecção de distração de motoristas (State Farm Distracted Driver Detection) do Kaggle. Congele os pesos até a última camada convolucional. Lembre-se de adicionar uma camada Flatten, e uma última camada que seja capaz de devolver a saída da classificação. Imprima a arquitetura do modelo.


In [13]:
def create_model(conv_base, first_trainable_layer='block5_conv3'): 
    
    model = Sequential()
    model.add(conv_base)
    
    set_trainable = False
    for layer in conv_base.layers:
        if layer.name == first_trainable_layer:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
            
    model.add(Flatten())
    model.add(Dense(units=10, activation='softmax'))
    
    return model
  

model = create_model(conv_base)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                81930     
Total params: 14,796,618
Trainable params: 2,441,738
Non-trainable params: 12,354,880
_________________________________________________________________


### Explique como foi calculado na arquitetura o número de parametros não treinados.

### Utilize 15% dos dados para treino e 85% para teste, considerando 0.001 como taxa de aprendizagem por 5 épocas. Para cada experimento reporte os gráficos de erro e acurácia do treino e validação. Utilize optimizador e função de erro de sua preferência de acordo com as disponíveis na documentação do Keras.

In [14]:
model.compile(optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc'])

W0716 18:06:37.627614 140166748837760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0716 18:06:37.659013 140166748837760 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
N_TRAIN = 19066
N_VAL = 3358

history = model.fit_generator(
    train_generator,
    steps_per_epoch=(N_TRAIN // BATCH_SIZE),
    epochs=5,
    validation_data=val_generator,
    validation_steps=(N_VAL // BATCH_SIZE) 
    )

Epoch 1/5
74/74 [==============================] - 641s 9s/step - loss: 2.7973 - acc: 0.8215 - val_loss: 2.8978 - val_acc: 0.8192
Epoch 2/5
31/74 [===========>..................] - ETA: 5:17 - loss: 2.8796 - acc: 0.8204

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, label='Training accuracy', marker='x')
plt.plot(epochs, val_acc, label='Validation accuracy', marker='o')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, label='Training loss', marker='x')
plt.plot(epochs, val_loss, label='Validation loss', marker='o')
plt.title('Training and validation loss')
plt.legend()

plt.show()